## VNET Data Preprocessing Test
This notebook was used for developing and testing VNET data preprocessing, after finishing, its contents have been copied to `vnet_preprocessing.py` file.

In [1]:
import numpy as np
import pandas as pd

from collections import defaultdict

pd.set_option('display.max_columns', None)

In [2]:
%cd ..

c:\Users\Goldy\Desktop\PRO projekty\workstuff\goldschmidt-playground\vnetwindow


In [3]:
data = pd.read_csv('data/48.flows', delimiter='|', header='infer')
windows = pd.read_csv('data/windows.csv')

In [4]:
data.head()

,IN_BYTES,IN_PKTS,PROTOCOL,TCP_FLAGS,L4_SRC_PORT,IPV4_SRC_ADDR,IPV6_SRC_ADDR,L4_DST_PORT,IPV4_DST_ADDR,IPV6_DST_ADDR,OUT_BYTES,OUT_PKTS,MIN_IP_PKT_LEN,MAX_IP_PKT_LEN,ICMP_TYPE,MIN_TTL,MAX_TTL,DIRECTION,FLOW_START_MILLISECONDS,FLOW_END_MILLISECONDS,SRC_FRAGMENTS,DST_FRAGMENTS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,SRC_TO_DST_AVG_THROUGHPUT,DST_TO_SRC_AVG_THROUGHPUT,NUM_PKTS_UP_TO_128_BYTES,NUM_PKTS_128_TO_256_BYTES,NUM_PKTS_256_TO_512_BYTES,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,NUM_PKTS_OVER_1514_BYTES,LONGEST_FLOW_PKT,SHORTEST_FLOW_PKT,RETRANSMITTED_IN_PKTS,RETRANSMITTED_OUT_PKTS,OOORDER_IN_PKTS,OOORDER_OUT_PKTS,DURATION_IN,DURATION_OUT,TCP_WIN_MIN_IN,TCP_WIN_MAX_IN,TCP_WIN_MSS_IN,TCP_WIN_SCALE_IN,TCP_WIN_MIN_OUT,TCP_WIN_MAX_OUT,TCP_WIN_MSS_OUT,TCP_WIN_SCALE_OUT,FLOW_VERDICT,SRC_TO_DST_IAT_MIN,SRC_TO_DST_IAT_MAX,SRC_TO_DST_IAT_AVG,SRC_TO_DST_IAT_STDDEV,DST_TO_SRC_IAT_MIN,DST_TO_SRC_IAT_MAX,DST_TO_SRC_IAT_AVG,DST_TO_SRC_IAT_STDDEV,APPLICATION_ID
0,32,1,17,0,46963,217.73.28.78,::,10001,18.195.24.157,::,0,0,0,32,0,0,0,1,1660934932096,1660934932096,0,0,0,0,256000,0,1,0,0,0,0,0,32,32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,369099017
1,52,1,6,16,80,81.89.49.98,::,9594,185.191.171.24,::,0,0,0,52,0,0,0,1,1660934932096,1660934932096,0,0,16,0,416000,0,1,0,0,0,0,0,52,52,0,0,0,0,0,0,235,235,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,369098759
2,64,1,17,0,58825,81.89.56.212,::,53,8.8.8.8,::,0,0,0,64,0,0,0,1,1660934932096,1660934932096,0,0,0,0,512000,0,1,0,0,0,0,0,64,64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,369098878
3,531,1,17,0,30300,86.110.248.222,::,30300,152.44.35.66,::,0,0,0,531,0,0,0,1,1660934932097,1660934932097,0,0,0,0,4248000,0,0,0,0,1,0,0,531,531,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,369098752
4,52,1,6,16,9001,46.229.238.172,::,34240,162.19.64.52,::,0,0,0,52,0,0,0,1,1660934932097,1660934932097,0,0,16,0,416000,0,1,0,0,0,0,0,52,52,0,0,0,0,0,0,501,501,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,369098752


In [62]:
data['DIRECTION'].value_counts()

0    142765
1     43650
Name: DIRECTION, dtype: int64

### Routines For Data Preprocessing

In [52]:
### Constants definitions
# Features not directly used for classification - provide additional information, but drop them
FEATURES_DROP = ['IPV4_SRC_ADDR', 'IPV6_SRC_ADDR',  # IPs are not directly useful, only as metadata
    'IPV4_DST_ADDR', 'IPV6_DST_ADDR',               # IPs are not directly useful, only as metadata
    'DIRECTION',                                    # Not useful for intrusion detection purposes
    'FLOW_VERDICT',                                 # These are all 0s
    'APPLICATION_ID'                                # Interesting metdata, but we cannot find an use for ID
    ]

# Features which were processed in the preprocessing function and are no longer needed for classification
FEATURES_DROP_PROCESSED = ['TCP_FLAGS', 'FLOW_START_MILLISECONDS', 'FLOW_END_MILLISECONDS', 'CLIENT_TCP_FLAGS',
    'SERVER_TCP_FLAGS']

# Categorical values to be encoded automatically
FEATURES_CATEGORICAL = ['PROTOCOL', 'L4_SRC_PORT', 'L4_DST_PORT']

# Numerical variables which should be scaled
FEATURES_NUMERICAL = ['IN_BYTES', 'IN_PKTS', 'OUT_BYTES', 'OUT_PKTS', 'MIN_IP_PKT_LEN', 'MAX_IP_PKT_LEN',
    'MIN_TTL', 'MAX_TTL', 'SRC_FRAGMENTS', 'DST_FRAGMENTS', 'SRC_TO_DST_AVG_THROUGHPUT', 'DST_TO_SRC_AVG_THROUGHPUT',
    'NUM_PKTS_UP_TO_128_BYTES', 'NUM_PKTS_128_TO_256_BYTES', 'NUM_PKTS_256_TO_512_BYTES', 'NUM_PKTS_512_TO_1024_BYTES',
    'NUM_PKTS_1024_TO_1514_BYTES', 'NUM_PKTS_OVER_1514_BYTES', 'LONGEST_FLOW_PKT', 'SHORTEST_FLOW_PKT',
    'RETRANSMITTED_IN_PKTS', 'RETRANSMITTED_OUT_PKTS', 'OOORDER_IN_PKTS', 'OOORDER_OUT_PKTS', 'DURATION_IN',
    'DURATION_OUT', 'TCP_WIN_MIN_IN', 'TCP_WIN_MAX_IN', 'TCP_WIN_MSS_IN', 'TCP_WIN_SCALE_IN', 'TCP_WIN_MIN_OUT',
    'TCP_WIN_MAX_OUT', 'TCP_WIN_MSS_OUT', 'TCP_WIN_SCALE_OUT', 'SRC_TO_DST_IAT_MIN', 'SRC_TO_DST_IAT_MAX',
    'SRC_TO_DST_IAT_AVG', 'SRC_TO_DST_IAT_STDDEV', 'DST_TO_SRC_IAT_MIN', 'DST_TO_SRC_IAT_MAX', 'DST_TO_SRC_IAT_AVG',
    'DST_TO_SRC_IAT_STDDEV']

FEATURES_NUMERICAL_COMPUTED = ['IN_BPS', 'IN_BPP', 'IN_PPS', 'OUT_BPS', 'OUT_BPP', 'OUT_PPS']

# Protocol to text data default dictionary
PROTO_MAPPER = defaultdict(lambda : 'OTHER', {
    1   : 'ICMP',
    6   : 'TCP',
    17  : 'UDP',
    47  : 'GRE',
    50  : 'ESP', 
    58  : 'ICMP6'
})

# Port categorization
# Sources:
#    - https://opensource.com/article/18/10/common-network-ports
#    - https://hostpapasupport.com/commonly-used-ports/
#    - https://en.wikipedia.org/wiki/List_of_TCP_and_UDP_port_numbers
#    - https://www.networkinghowtos.com/howto/common-vpn-ports-and-protocols
#    - https://www.rfc-editor.org/rfc/rfc7194.html 
#    - https://en.wikipedia.org/wiki/Denial-of-service_attack#Amplification
#    - https://manuals.playstation.net/document/en/psvita/psn/firewall.html
#    - https://www.speedguide.net/ports.php

PCAT_NOPORT      = {0}                                                           # No TCP/UDP port used
PCAT_NOENC_WEB   = {80, 8080}                                                    # Web traffic
PCAT_TLS         = {443}                                                         # TLS traffic, probably web
PCAT_DNS         = {53}                                                          # Separate category due to traffic volume
PCAT_EMAIL       = {25, 26, 109, 110, 143, 209, 218, 220, 587, 993, 2095, 2096}  # Email traffic
PCAT_VPN         = {500, 1194, 1701, 1723, 4500}                                 # VPNs like L2TP, IPSec, OpenVPN...
PCAT_DATA        = {20, 21, 22, 69, 115, 989, 990, 2077, 2078}                   # Data transfer FTP, STFP, TFTP
PCAT_SHELL       = {22, 23, 513, 514}                                            # Shell services - SSH, RSH,...
PCAT_PLAYSTATION = {465, 3478, 3479, 3480}                                       # Playstation Network
PCAT_CHAT        = {194, 517, 518, 2351, 6667, 6697}                             # IRC chat
PCAT_QUERY       = {17, 19, 123, 1900, 3283, 4462, 4463, 5683, 6881, 6882,       # Query-response for amplification
                    6883, 6883, 6884, 6885, 6886, 6887, 6888, 6889, 11211, 26000}

# In order to make port categorization computationally easier, create a search dict from the above sets
PCAT_SEARCHDICT = defaultdict(lambda: 'OTHER', dict.fromkeys(PCAT_NOENC_WEB, 'WEB_NOENC') | dict.fromkeys(PCAT_TLS, 'TLS') | \
    dict.fromkeys(PCAT_DNS, 'DNS') | dict.fromkeys(PCAT_EMAIL, 'EMAIL') | dict.fromkeys(PCAT_VPN, 'VPN') | \
    dict.fromkeys(PCAT_DATA, 'DATA') | dict.fromkeys(PCAT_SHELL, 'SHELL') | \
    dict.fromkeys(PCAT_PLAYSTATION, 'PLAYSTATION') | dict.fromkeys(PCAT_CHAT, 'CHAT') | dict.fromkeys(PCAT_QUERY, 'QUERY'))

In [56]:
from sklearn.preprocessing import MinMaxScaler

### Data preprocessing functions
def map_tcp_flags(tcp_flags_ser : pd.Series, col_prefix: str) -> pd.DataFrame: 
    """Map a series of TCP flags into DataFrame, with most important flags parsed into separate columns."""
    
    col_prefix = col_prefix + '_' if col_prefix else col_prefix
    colnames = [col_prefix + 'FLAG_' + colname for colname in ['FIN', 'SYN', 'RST', 'PSH', 'ACK', 'URG', 'ECE', 'CWR']]
    deenc_flags_df = pd.DataFrame(np.zeros((len(tcp_flags_ser), 8), dtype=np.int8), columns=colnames)

    for row_idx in range(0, len(tcp_flags_ser)):
        for flag_idx in range(8):
            flag_value = 1 << flag_idx
            if (tcp_flags_ser.iloc[row_idx] & flag_value) != 0:
                deenc_flags_df.iloc[row_idx, flag_idx] = 1

    return deenc_flags_df


def categorize_port(port : int) -> str:
    """Performs port categorization into one of 12 pre-specified groups"""

    portcat = PCAT_SEARCHDICT[port]

    # Perform correction to other special, dynamic port category
    # Dynamic ports are defined from 49152 to 65535, since ports are 16-bit, upper bound is not necessary
    if portcat == 'OTHER' and port >= 49152:
        portcat == 'DYNAMIC'

    return portcat


def vnet_preprocess(data : pd.DataFrame) -> pd.DataFrame:
    """Preprocesses dataset exported by nProbe into ML-acceptable form."""
    data = data.copy(deep=True)

    # Compute "normalized" flow durations for each side in seconds or future fields computation
    # This means that if the duration is 0 (0 or 1 packet sent), it it set to 1ms to allow divisions
    data['IN_DUR_S']  = data['DURATION_IN'].apply(lambda x : x / 1000.0 if x > 0 else 0.001)
    data['OUT_DUR_S'] = data['DURATION_OUT'].apply(lambda x : x / 1000.0 if x > 0 else 0.001)

    # Map protocols into their respective names to limit their number for one-hot encoding
    data['PROTOCOL'] = data['PROTOCOL'].apply(lambda x : PROTO_MAPPER[x])

    # Map TCP flags
    data = pd.concat([data, map_tcp_flags(data['TCP_FLAGS'], '')], axis=1)
    data = pd.concat([data, map_tcp_flags(data['CLIENT_TCP_FLAGS'], 'CLIENT')], axis=1)
    data = pd.concat([data, map_tcp_flags(data['SERVER_TCP_FLAGS'], 'SERVER')], axis=1)

    # Perform port categorization
    data['L4_SRC_PORT'] = data['L4_SRC_PORT'].apply(categorize_port)
    data['L4_DST_PORT'] = data['L4_DST_PORT'].apply(categorize_port)

    # Decode ICMP types and codes
    data['ICMP_CODE'] = data['ICMP_TYPE'] % 256
    data['ICMP_TYPE'] = data['ICMP_TYPE'] // 256

    # Compute other statistics
    data['IN_BPS'] = data['IN_BYTES'] / data['IN_DUR_S']    # Client's bytes-per-second
    data['IN_BPP'] = data['IN_BYTES'] / data['IN_PKTS']     # Client's bytes-per-packet
    data['IN_PPS'] = data['IN_PKTS'] / data['IN_DUR_S']     # Client's packets-per-second
    data['OUT_BPS'] = data['OUT_BYTES'] / data['OUT_DUR_S'] # Server's bytes-per-second
    data['OUT_BPP'] = data['OUT_BYTES'] / data['OUT_PKTS']  # Server's bytes-per-packet
    data['OUT_PPS'] = data['OUT_PKTS'] / data['OUT_DUR_S']  # Server's bytes-per-second

    # Fix bytes-per-packet values if number of packets was 0. This is a pretty ugly
    # principle, as many divisions by 0 occur. Maybe the computation itself would need fix
    data['IN_BPP']  = data['IN_BPP'].apply(lambda x : 0 if np.isnan(x) else x)
    data['OUT_BPP'] = data['OUT_BPP'].apply(lambda x : 0 if np.isnan(x) else x)
    
    # Perform one-hot encoding of categorical variables
    data = pd.get_dummies(data, columns=FEATURES_CATEGORICAL)

    # Scale existing and newly computed numerical variables
    features_to_scale = FEATURES_NUMERICAL + FEATURES_NUMERICAL_COMPUTED

    data[features_to_scale] = MinMaxScaler().fit_transform(data[features_to_scale])

    # Drop features which cannot be used for classification and other auxilliary features
    data = data.drop(columns=FEATURES_DROP + FEATURES_DROP_PROCESSED)
    data = data.drop(columns=['IN_DUR_S', 'OUT_DUR_S'])

    return data

In [57]:
datax = vnet_preprocess(data)

In [58]:
datax.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,MIN_IP_PKT_LEN,MAX_IP_PKT_LEN,ICMP_TYPE,MIN_TTL,MAX_TTL,SRC_FRAGMENTS,DST_FRAGMENTS,SRC_TO_DST_AVG_THROUGHPUT,DST_TO_SRC_AVG_THROUGHPUT,NUM_PKTS_UP_TO_128_BYTES,NUM_PKTS_128_TO_256_BYTES,NUM_PKTS_256_TO_512_BYTES,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,NUM_PKTS_OVER_1514_BYTES,LONGEST_FLOW_PKT,SHORTEST_FLOW_PKT,RETRANSMITTED_IN_PKTS,RETRANSMITTED_OUT_PKTS,OOORDER_IN_PKTS,OOORDER_OUT_PKTS,DURATION_IN,DURATION_OUT,TCP_WIN_MIN_IN,TCP_WIN_MAX_IN,TCP_WIN_MSS_IN,TCP_WIN_SCALE_IN,TCP_WIN_MIN_OUT,TCP_WIN_MAX_OUT,TCP_WIN_MSS_OUT,TCP_WIN_SCALE_OUT,SRC_TO_DST_IAT_MIN,SRC_TO_DST_IAT_MAX,SRC_TO_DST_IAT_AVG,SRC_TO_DST_IAT_STDDEV,DST_TO_SRC_IAT_MIN,DST_TO_SRC_IAT_MAX,DST_TO_SRC_IAT_AVG,DST_TO_SRC_IAT_STDDEV,FLAG_FIN,FLAG_SYN,FLAG_RST,FLAG_PSH,FLAG_ACK,FLAG_URG,FLAG_ECE,FLAG_CWR,CLIENT_FLAG_FIN,CLIENT_FLAG_SYN,CLIENT_FLAG_RST,CLIENT_FLAG_PSH,CLIENT_FLAG_ACK,CLIENT_FLAG_URG,CLIENT_FLAG_ECE,CLIENT_FLAG_CWR,SERVER_FLAG_FIN,SERVER_FLAG_SYN,SERVER_FLAG_RST,SERVER_FLAG_PSH,SERVER_FLAG_ACK,SERVER_FLAG_URG,SERVER_FLAG_ECE,SERVER_FLAG_CWR,ICMP_CODE,IN_BPS,IN_BPP,IN_PPS,OUT_BPS,OUT_BPP,OUT_PPS,PROTOCOL_ESP,PROTOCOL_GRE,PROTOCOL_ICMP,PROTOCOL_ICMP6,PROTOCOL_OTHER,PROTOCOL_TCP,PROTOCOL_UDP,L4_SRC_PORT_CHAT,L4_SRC_PORT_DATA,L4_SRC_PORT_DNS,L4_SRC_PORT_EMAIL,L4_SRC_PORT_OTHER,L4_SRC_PORT_PLAYSTATION,L4_SRC_PORT_QUERY,L4_SRC_PORT_SHELL,L4_SRC_PORT_TLS,L4_SRC_PORT_VPN,L4_SRC_PORT_WEB_NOENC,L4_DST_PORT_CHAT,L4_DST_PORT_DATA,L4_DST_PORT_DNS,L4_DST_PORT_EMAIL,L4_DST_PORT_OTHER,L4_DST_PORT_PLAYSTATION,L4_DST_PORT_QUERY,L4_DST_PORT_SHELL,L4_DST_PORT_TLS,L4_DST_PORT_VPN,L4_DST_PORT_WEB_NOENC
0,1.224280e-07,0.0,0.0,0.0,0.0,0.002898,0,0.0,0.0,0.0,0.0,0.000494,0.0,0.000052,0.0,0.0,0.000000,0.0,0.0,0.002898,0.005113,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000494,0.008108,0.02082,0.0,0.0,0.0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,3.264746e-07,0.0,0.0,0.0,0.0,0.007728,0,0.0,0.0,0.0,0.0,0.000803,0.0,0.000052,0.0,0.0,0.000000,0.0,0.0,0.007728,0.013634,0.0,0.0,0.0,0.0,0.0,0.0,0.003586,0.003586,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.000803,0.021622,0.02082,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
2,4.489025e-07,0.0,0.0,0.0,0.0,0.010625,0,0.0,0.0,0.0,0.0,0.000989,0.0,0.000052,0.0,0.0,0.000000,0.0,0.0,0.010625,0.018747,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000988,0.029730,0.02082,0.0,0.0,0.0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,5.213391e-06,0.0,0.0,0.0,0.0,0.123400,0,0.0,0.0,0.0,0.0,0.008204,0.0,0.000000,0.0,0.0,0.000199,0.0,0.0,0.123400,0.217725,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.008204,0.345270,0.02082,0.0,0.0,0.0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,3.264746e-07,0.0,0.0,0.0,0.0,0.007728,0,0.0,0.0,0.0,0.0,0.000803,0.0,0.000052,0.0,0.0,0.000000,0.0,0.0,0.007728,0.013634,0.0,0.0,0.0,0.0,0.0,0.0,0.007645,0.007645,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.000803,0.021622,0.02082,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [61]:
datax.describe()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,MIN_IP_PKT_LEN,MAX_IP_PKT_LEN,ICMP_TYPE,MIN_TTL,MAX_TTL,SRC_FRAGMENTS,DST_FRAGMENTS,SRC_TO_DST_AVG_THROUGHPUT,DST_TO_SRC_AVG_THROUGHPUT,NUM_PKTS_UP_TO_128_BYTES,NUM_PKTS_128_TO_256_BYTES,NUM_PKTS_256_TO_512_BYTES,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,NUM_PKTS_OVER_1514_BYTES,LONGEST_FLOW_PKT,SHORTEST_FLOW_PKT,RETRANSMITTED_IN_PKTS,RETRANSMITTED_OUT_PKTS,OOORDER_IN_PKTS,OOORDER_OUT_PKTS,DURATION_IN,DURATION_OUT,TCP_WIN_MIN_IN,TCP_WIN_MAX_IN,TCP_WIN_MSS_IN,TCP_WIN_SCALE_IN,TCP_WIN_MIN_OUT,TCP_WIN_MAX_OUT,TCP_WIN_MSS_OUT,TCP_WIN_SCALE_OUT,SRC_TO_DST_IAT_MIN,SRC_TO_DST_IAT_MAX,SRC_TO_DST_IAT_AVG,SRC_TO_DST_IAT_STDDEV,DST_TO_SRC_IAT_MIN,DST_TO_SRC_IAT_MAX,DST_TO_SRC_IAT_AVG,DST_TO_SRC_IAT_STDDEV,FLAG_FIN,FLAG_SYN,FLAG_RST,FLAG_PSH,FLAG_ACK,FLAG_URG,FLAG_ECE,FLAG_CWR,CLIENT_FLAG_FIN,CLIENT_FLAG_SYN,CLIENT_FLAG_RST,CLIENT_FLAG_PSH,CLIENT_FLAG_ACK,CLIENT_FLAG_URG,CLIENT_FLAG_ECE,CLIENT_FLAG_CWR,SERVER_FLAG_FIN,SERVER_FLAG_SYN,SERVER_FLAG_RST,SERVER_FLAG_PSH,SERVER_FLAG_ACK,SERVER_FLAG_URG,SERVER_FLAG_ECE,SERVER_FLAG_CWR,ICMP_CODE,IN_BPS,IN_BPP,IN_PPS,OUT_BPS,OUT_BPP,OUT_PPS,PROTOCOL_ESP,PROTOCOL_GRE,PROTOCOL_ICMP,PROTOCOL_ICMP6,PROTOCOL_OTHER,PROTOCOL_TCP,PROTOCOL_UDP,L4_SRC_PORT_CHAT,L4_SRC_PORT_DATA,L4_SRC_PORT_DNS,L4_SRC_PORT_EMAIL,L4_SRC_PORT_OTHER,L4_SRC_PORT_PLAYSTATION,L4_SRC_PORT_QUERY,L4_SRC_PORT_SHELL,L4_SRC_PORT_TLS,L4_SRC_PORT_VPN,L4_SRC_PORT_WEB_NOENC,L4_DST_PORT_CHAT,L4_DST_PORT_DATA,L4_DST_PORT_DNS,L4_DST_PORT_EMAIL,L4_DST_PORT_OTHER,L4_DST_PORT_PLAYSTATION,L4_DST_PORT_QUERY,L4_DST_PORT_SHELL,L4_DST_PORT_TLS,L4_DST_PORT_VPN,L4_DST_PORT_WEB_NOENC
count,1.864150e+05,186415.000000,1.864150e+05,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.0,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.0,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.0,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000,186415.000000
mean,1.373902e-04,0.000273,1.437386e-04,0.000200,0.015849,0.086248,5.969761,0.145175,0.145743,0.000024,0.000012,0.000916,0.000849,0.000726,0.000253,0.000681,0.000259,0.000217,0.000013,0.086248,0.021807,0.000335,0.000166,0.000114,0.000076,0.155901,0.108889,0.175763,0.176691,0.031854,0.099994,0.100271,0.100517,0.020285,0.058682,0.032841,0.083589,0.044774,0.036581,0.013046,0.054228,0.021775,0.028250,0.138369,0.255441,0.086833,0.367873,0.468138,0.0,0.010402,0.010305,0.127801,0.253826,0.057544,0.357005,0.448220,0.0,0.009833,0.009307,0.086565,0.124518,0.044669,0.246638,0.395027,0.0,0.003841,0.001100,0.035861,0.001276,0.093645,0.014287,0.000965,0.104900,0.004013,0.002875,0.001084,0.038720,0.000139,0.000687,0.557793,0.398702,0.000027,0.000703,0.004490,0.019982,0.867548,0.000488,0.001819,0.002848,0.085186,0.005552,0.011356,0.000048,0.001287,0.142891,0.021425,0.618453,0.001298,0.005874,0.014688,0.159708,0.006346,0.027981
std,3.46003

Preprocessing notes and ideas
- Durations are often too short or 0s, normalizing to 1ms if duration is 0 is used for proper computations
- Throughput computations are not equal to bps computations. What are throughput computaitions after all?
- Completely dropped features:
   - `IPV4_SRC_ADDR`, `IPV6_SRC_ADDR`, `IPV4_DST_ADDR`, `IPV6_DST_ADDR` (IPs are not directly used for classification, only for windowing)
   - `DIRECTION` (currently only considered for metadata, not seeing usage for ID)
   - `FLOW_VERDICT` (all 0s, not informational)
   - `APPLICATION_ID` (interesting, but not currently seeing into what the value represents)
    